In [49]:
from os import path, makedirs, rename, remove
from time import time
import argparse
import pickle

import numpy as np
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer

In [50]:
train_data = pd.read_csv('./data_fake_news/train.csv')
test_data = pd.read_csv('./data_fake_news/test.csv')

In [51]:
test_data.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [52]:
descriptions = train_data['text']
train_size = int(len(descriptions) * .8)
labels = train_data['label']
train_description = descriptions[:train_size].astype('str')
train_labels = labels[:train_size]
test_description = descriptions[train_size:].astype('str')
test_labels = labels[train_size:]

In [53]:
top_labels = ['0', '1']
description_embeddings = hub.text_embedding_column(
  "text", 
  module_spec="https://tfhub.dev/google/universal-sentence-encoder/2"
)

In [54]:
encoder = LabelBinarizer()
encoder.fit_transform(train_labels)
train_encoded = encoder.transform(train_labels)
test_encoded = encoder.transform(test_labels)
num_classes = len(encoder.classes_)

In [55]:
binary_label_head = tf.contrib.estimator.binary_classification_head(
                        loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
                    )

In [56]:
estimator = tf.contrib.estimator.DNNEstimator(
    head=binary_label_head,
    hidden_units=[64,10],
    feature_columns=[description_embeddings]
)

W0328 11:45:20.859363 24732 tf_logging.py:126] Using temporary folder as model directory: C:\Users\I349732\AppData\Local\Temp\tmp8q57kksq


In [57]:
features = {
  "text": np.array(train_description).astype(np.str)
}
labels = np.array(train_encoded).astype(np.int32)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    features, 
    labels, 
    shuffle=True, 
    batch_size=32, 
    num_epochs=20
)

In [59]:
tf.logging.set_verbosity(tf.logging.INFO)
estimator.train(input_fn=train_input_fn)

I0328 11:50:57.155110 24732 tf_logging.py:116] Calling model_fn.
I0328 11:51:12.783882 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_0:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with Embeddings_en/sharded_0
I0328 11:51:12.792905 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_1:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with Embeddings_en/sharded_1
I0328 11:51:12.803934 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_10:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a3847

I0328 11:51:13.100725 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_0/weights:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_0/weights
I0328 11:51:13.113758 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_1/bias:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_1/bias
I0328 11:51:13.127796 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_

I0328 12:05:28.308457 24732 tf_logging.py:116] loss = 0.229186, step = 2302 (29.506 sec)
I0328 12:06:02.903590 24732 tf_logging.py:116] global_step/sec: 2.88927
I0328 12:06:02.920440 24732 tf_logging.py:116] loss = 0.297893, step = 2402 (34.612 sec)
I0328 12:06:35.780025 24732 tf_logging.py:116] global_step/sec: 3.04169
I0328 12:06:35.780025 24732 tf_logging.py:116] loss = 0.212232, step = 2502 (32.860 sec)
I0328 12:07:09.640931 24732 tf_logging.py:116] global_step/sec: 2.95326
I0328 12:07:09.645945 24732 tf_logging.py:116] loss = 0.296238, step = 2602 (33.865 sec)
I0328 12:07:39.412388 24732 tf_logging.py:116] global_step/sec: 3.35892
I0328 12:07:39.417402 24732 tf_logging.py:116] loss = 0.193219, step = 2702 (29.772 sec)
I0328 12:08:07.473302 24732 tf_logging.py:116] global_step/sec: 3.56368
I0328 12:08:07.486291 24732 tf_logging.py:116] loss = 0.178561, step = 2802 (28.069 sec)
I0328 12:08:38.973023 24732 tf_logging.py:116] global_step/sec: 3.17463
I0328 12:08:38.978037 24732 tf_log

KeyboardInterrupt: 

In [62]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"text": np.array(test_description).astype(np.str)}, test_encoded.astype(np.int32), shuffle=False)

estimator.evaluate(input_fn=eval_input_fn)

I0328 12:13:57.589858 24732 tf_logging.py:116] Calling model_fn.
I0328 12:14:23.532740 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_0:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with Embeddings_en/sharded_0
I0328 12:14:23.541803 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_1:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with Embeddings_en/sharded_1
I0328 12:14:23.550834 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/Embeddings_en/sharded_10:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a3847

I0328 12:14:23.833539 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_1/bias:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_1/bias
I0328 12:14:23.846573 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_1/weights:0 from checkpoint b'C:\\Users\\I349732\\AppData\\Local\\Temp\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47\\variables\\variables' with SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_1/weights
I0328 12:14:23.858606 24732 tf_logging.py:116] Initialize variable dnn/input_from_feature_columns/input_layer/text_hub_module_embedding/module/SHARED_RANK_ANSWER/response_encoder_0/tanh_layer_

{'accuracy': 0.88149041,
 'accuracy_baseline': 0.50793266,
 'auc': 0.95476878,
 'auc_precision_recall': 0.95874542,
 'average_loss': 0.27444336,
 'global_step': 3609,
 'label/mean': 0.50793266,
 'loss': 0.27423015,
 'prediction/mean': 0.48924395}